# Regression



In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import scipy.io
import os
import numpy as np
from sklearn.metrics import mean_squared_error, make_scorer

In [20]:
path_code = os.getcwd()
bbp_dataset = scipy.io.loadmat(os.path.join(path_code, "../Data/bbp_regr.mat"))

In [43]:
ae = bbp_dataset["ae"].T
conf = bbp_dataset["conf"].T
wer = bbp_dataset["wer"].T
y = bbp_dataset["slp"].T

X = np.column_stack((ae, conf, wer))

In [44]:
X_converted = []
for row in X:
    converted_row = [np.array(array) if isinstance(array, (list, tuple)) else array for array in row]
    X_converted.append(converted_row)

# Estrarre e concatenare gli array interni di X
X_concatenated = []
for row in X_converted:
    concatenated_row = np.hstack([array.flatten() for array in row])
    X_concatenated.append(concatenated_row)

X_concatenated = np.array(X_concatenated)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (17,) + inhomogeneous part.

In [34]:
type(X)

list

In [7]:
loo=LeaveOneOut()

In [10]:
y_true = []
y_pred = []

In [12]:
model = SVR(kernel='linear')
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5, 1]
}

In [31]:
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Modello SVR
    svr = SVR(kernel='linear')

    # Grid search con LOOCV interno per selezionare i migliori parametri
    inner_loo = LeaveOneOut()
    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=inner_loo, scoring='neg_mean_squared_error', error_score='raise')
    grid_search.fit(X_train, y_train)

    # Migliori parametri trovati nella grid search interna
    best_params = grid_search.best_params_
    best_svr = grid_search.best_estimator_

    # Fit del modello con i migliori parametri sul training set esterno e predizione
    best_svr.fit(X_train, y_train)
    y_pred.append(best_svr.predict(X_test)[0])
    y_true.append(y_test[0])

# Calcolare l'RMSE sui risultati finali
final_rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'Nested LOOCV RMSE: {final_rmse:.2f}')

TypeError: only integer scalar arrays can be converted to a scalar index

NameError: name 'shape' is not defined